Ta koda poišče graf z najmanjšim ali največjim številom podpoti iz random grafa z podanim številom vozlišč [n] in ciklomatičnim št. [c].
Uporablja optimizirano funkcijo za subpath_number.

In [ ]:
import random as pyrandom
from sage.all import *


# ============================================================
#   HITRA VERZIJA SUBPATH_NUMBER (KLJUČNA OPTIMIZACIJA)
# ============================================================

def subpath_number(G):
    """
    Optimizirana verzija:
    - Za vsak start vertex s naredi en DFS.
    - V enem DFS-u prešteje vse enostavne poti s → v za VSE v.
    - Ne shranjuje poti, samo inkrementira števce.
    - Eksponentna eksplozija je drastično manjša.
    """

    V = list(G.vertices())
    idx = {v: i for i, v in enumerate(V)}
    n = len(V)

    counts = [[0] * n for _ in range(n)]

    for s in V:
        s_idx = idx[s]

        def dfs(u, visited):
            u_idx = idx[u]
            counts[s_idx][u_idx] += 1     # registriramo pot s → u

            for nei in G.neighbors(u):
                if nei not in visited:
                    visited.add(nei)
                    dfs(nei, visited)
                    visited.remove(nei)

        visited = set([s])
        dfs(s, visited)

    total = 0
    for i in range(n):
        for j in range(i, n):
            total += counts[i][j]

    return total



# ============================================================
#   Mutacija grafa
# ============================================================

def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H



# ============================================================
#   Naključni povezan graf G(n, m)
# ============================================================

def random_connected_graph(n, m):
    while True:
        G = graphs.RandomGNM(n, m)
        if G.is_connected():
            return G



# ============================================================
#   Simulated annealing + STOP WHEN PTC FOUND
# ============================================================

def simulated_annealing(n, m, direction,
                        T_start=3, T_end=0.001,
                        cooling=0.99, max_steps=1000):

    G = random_connected_graph(n, m)

    best_G = G.copy()
    best_score = subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):

        new_G = mutate_graph(current_G)
        new_score = subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G = new_G
            current_score = new_score
        else:
            if pyrandom.random() < exp(-delta / T):
                current_G = new_G
                current_score = new_score

        improved = False

        if direction == "min":
            if current_score < best_score:
                improved = True
        else:
            if current_score > best_score:
                improved = True

        if improved:
            best_score = current_score
            best_G = current_G.copy()

            print(f"Improved at step {step}: best = {best_score}")
            best_G.show()

            if direction == "min" and is_ptc(best_G):
                print("PTC FOUND → stopping SA early!")
                return best_G, best_score

        T *= cooling
        if T < T_end:
            break

    return best_G, best_score



# ============================================================
#   TEST
# ============================================================

n = 9
c = 20
m = c + n - 1
direction = "min"

best_graph, best_value = simulated_annealing(n, m, direction)
print("Best:", best_value)
best_graph.show()


To je ista funkcija kot prejšnja, le da je še narejena zanka po intervalu št. povezav

In [ ]:
import random as pyrandom
from sage.all import *

# ============================================================
#   Helper: all simple paths + subpath number
# ============================================================

def all_simple_paths(G, start, end, visited=None):
    if visited is None:
        visited = []
    visited = visited + [start]
    if start == end:
        return [visited]
    paths = []
    for nei in G.neighbors(start):
        if nei not in visited:
            newp = all_simple_paths(G, nei, end, visited)
            paths.extend(newp)
    return paths

def subpath_number(G):
    V = G.vertices()
    total = 0
    for i in range(len(V)):
        for j in range(i, len(V)):
            total += len(all_simple_paths(G, V[i], V[j]))
    return total


# ============================================================
#   Mutacija grafa
# ============================================================

def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H


# ============================================================
#   Naključni povezan graf G(n, m)
# ============================================================

def random_connected_graph(n, m):
    while True:
        G = graphs.RandomGNM(n, m)
        if G.is_connected():
            return G


# ============================================================
#   Preverjanje PTC
# ============================================================

def is_ptc(G):
    if not G.is_connected():
        return False

    blocks, cut_vertices = G.blocks_and_cut_vertices()
    nontrivial = [Bl for Bl in blocks if len(Bl) >= 2]

    if len(nontrivial) < 2:
        return False

    # vsi nenetrivialni bloki morajo biti trikotniki
    for Bl in nontrivial:
        sub = G.subgraph(Bl)
        if not (sub.order() == 3 and sub.size() == 3):
            return False

    idx = list(range(len(nontrivial)))
    adj = {i: [] for i in idx}

    # povezave med trikotniki
    for i in idx:
        for j in idx:
            if j <= i:
                continue
            inter = set(nontrivial[i]).intersection(nontrivial[j])
            if len(inter) > 1:
                return False
            if len(inter) == 1:
                adj[i].append(j)
                adj[j].append(i)

    deg = [len(adj[i]) for i in idx]

    if any(d == 0 or d > 2 for d in deg):
        return False

    ends = sum(1 for d in deg if d == 1)
    mids = sum(1 for d in deg if d == 2)

    if ends != 2:
        return False
    if ends + mids != len(idx):
        return False

    # preveri povezavnost verige trikotnikov
    start = idx[0]
    visited = {start}
    stack = [start]

    while stack:
        v = stack.pop()
        for w in adj[v]:
            if w not in visited:
                visited.add(w)
                stack.append(w)

    return len(visited) == len(idx)


# ============================================================
#   Simulated annealing (BREZ izrisovanja vmesnih izboljšav)
# ============================================================

def simulated_annealing(n, m, direction,
                        T_start=5.0, T_end=0.01,
                        cooling=0.999, max_steps=2000):

    G = random_connected_graph(n, m)

    best_G = G.copy()
    best_score = subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):

        new_G = mutate_graph(current_G)
        new_score = subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G, current_score = new_G, new_score
        else:
            if pyrandom.random() < exp(-delta / T):
                current_G, current_score = new_G, new_score

        # preveri izboljšavo
        improved = (
            (direction == "min" and current_score < best_score) or
            (direction == "max" and current_score > best_score)
        )

        if improved:
            best_score = current_score
            best_G = current_G.copy()

        T *= cooling
        if T < T_end:
            break

    return best_G, best_score


# ============================================================
#   ZANKA PO INTERVALU m
# ============================================================

n = 8
m_start = 11
m_end   = 13
direction = "min"   # ali "max"

results = []

for m in range(m_start, m_end + 1):
    print(f"\n=== n={n}, m={m} ===")
    best_graph, best_value = simulated_annealing(n, m, direction)
    print(f"Optimalni p_n(G) = {best_value}")
    results.append((m, best_value, best_graph))


# ------------------------------------------------------------
#   Če želiš izrisati samo končne grafe:
# ------------------------------------------------------------

for m, val, G in results:
    print(f"\nKončni graf za m={m}, p_n(G)={val}")
    G.show()


Funkcija, ki izriše graf ko podaš ime

In [ ]:
from sage.all import *

g6 = "GCOf?w"   # tukaj daš svoj graph6 zapis
G = Graph(g6)  # iz enoličnega zapisa narediš graf

G.show()       # odpre okno / zriše graf
# ali:
plot(G)        # vrne sliko grafa kot objekt

Ta funkcija ti za n = 9 in vsa možna ciklomatska števila, poišče z SA dobre približke za ekstreme grafov (grafe z najmanjšim in največjim številom podpoti). <br>
Koda posodablja vrstice, če jo šeenkrat zaženeš. <br>
Točni podatki za grafe do n =8 so odvzeti iz datoteke 1del_mali.csv, ta funkcija pa rezultate zapisuje v 1del_SA_n_9.csv <br>
Tako za max kot za min je posebej koda runnala približno 35 minut. <br>
Ne shranjuje slik, shranjuje samo imena. <br>

In [ ]:

import random as pyrandom
from sage.all import *
import pandas as pd
import os
import time

# ============================================================
#  SUBPATH NUMBER (OPTIMIZIRANO)
# ============================================================

def subpath_number(G):
    V = list(G.vertices())
    idx = {v: i for i, v in enumerate(V)}
    n = len(V)

    counts = [[0] * n for _ in range(n)]

    for s in V:
        s_idx = idx[s]

        def dfs(u, visited):
            u_idx = idx[u]
            counts[s_idx][u_idx] += 1

            for nei in G.neighbors(u):
                if nei not in visited:
                    visited.add(nei)
                    dfs(nei, visited)
                    visited.remove(nei)

        visited = set([s])
        dfs(s, visited)

    total = 0
    for i in range(n):
        for j in range(i, n):
            total += counts[i][j]

    return total


# ============================================================
#  SHARANJE OPTIMUMA v CSV (popolnoma popravljeno)
# ============================================================

def save_to_csv(n, mu, graph, score, direction):
    filename = "1del_SA_n_9.csv"

    g6 = graph.graph6_string()

    # Če CSV ne obstaja → kreiraj
    if not os.path.exists(filename):
        df = pd.DataFrame(columns=[
            "n",
            "µ(G)",
            "min p_n(G)",
            "max p_n(G)",
            "graph6_min",
            "graph6_max"
        ])
        df.to_csv(filename, index=False)

    df = pd.read_csv(filename, encoding="utf-8")

    # poskrbimo, da stolpci obstajajo (če imaš kak star CSV)
    for col in ["min p_n(G)", "max p_n(G)", "graph6_min", "graph6_max"]:
        if col not in df.columns:
            df[col] = pd.NA

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    n_int = int(n)
    mu_int = int(mu)

    mask = (df["n"] == n_int) & (df["µ(G)"] == mu_int)

    # 1) Če vrstice NI → nova
    if not mask.any():
        new_row = {
            "n": n_int,
            "µ(G)": mu_int,
            "min p_n(G)": score if direction == "min" else pd.NA,
            "max p_n(G)": score if direction == "max" else pd.NA,
            "graph6_min": g6 if direction == "min" else pd.NA,
            "graph6_max": g6 if direction == "max" else pd.NA
        }

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        df.to_csv(filename, index=False)
        return

    # 2) Če vrstica OBSTAJA → update
    idx = df.index[mask].tolist()[0]

    if direction == "min":
        old = df.at[idx, "min p_n(G)"]

        if pd.isna(old) or old == "":
            # še nič ni zapisano → zapišemo
            df.at[idx, "min p_n(G)"] = score
            df.at[idx, "graph6_min"] = g6
        else:
            # že obstaja številka → zamenjamo samo, če je nova boljša (manjša)
            if score < float(old):
                df.at[idx, "min p_n(G)"] = score
                df.at[idx, "graph6_min"] = g6

    else:  # direction == "max"
        old = df.at[idx, "max p_n(G)"]

        if pd.isna(old) or old == "":
            df.at[idx, "max p_n(G)"] = score
            df.at[idx, "graph6_max"] = g6
        else:
            if score > float(old):
                df.at[idx, "max p_n(G)"] = score
                df.at[idx, "graph6_max"] = g6

    df.to_csv(filename, index=False)



# ============================================================
#  MUTATE GRAPH
# ============================================================

def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H


# ============================================================
#  DODAJ 1 VOZLIŠČE
# ============================================================

def add_one_vertex_and_connect(G_old, direction):
    H = Graph(G_old)

    verts = H.vertices()
    new_v = max(verts) + 1
    H.add_vertex(new_v)

    degrees = {v: H.degree(v) for v in H.vertices() if v != new_v}

    if direction == "min":
        anchor = min(degrees, key=degrees.get)
    else:
        anchor = max(degrees, key=degrees.get)

    H.add_edge(anchor, new_v)
    return H


# ============================================================
#  DODAJ 1 POVEZAVO (najbližji non-edge)
# ============================================================

def add_one_edge(G_old):
    H = Graph(G_old)

    V = H.vertices()
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if not H.has_edge(u, v):
                H.add_edge(u, v)
                return H

    raise ValueError("Graf je že poln.")


# ============================================================
#  LOAD G(n,µ) IZ TVOJEGA CSV — POPRAVLJENO
# ============================================================

def load_G(n_val, mu_val, direction):
    df = pd.read_csv("1del_SA_n_9.csv", encoding="utf-8")

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    # Konverzija Sage Integer → Python int
    n_val = int(n_val)
    mu_val = int(mu_val)

    subset = df[df["n"] == n_val]
    if subset.empty:
        raise ValueError(f"Ni vrstic za n={n_val}")

    row = subset[subset["µ(G)"] == mu_val]
    if row.empty:
        raise ValueError(f"Ni vrstice za n={n_val}, µ={mu_val}")

    # KLJUČNO — prisili Python int indeks
    row = row.iloc[int(0)]      # ← EDINA PRAVA OBLIKA

    # graph6
    if direction == "min":
        g6 = str(row["graph6_min"]).strip()
    else:
        g6 = str(row["graph6_max"]).strip()

    if g6 == "":
        raise ValueError(f"Prazni graph6 zapis pri n={n_val}, µ={mu_val}")

    return Graph(g6)



# ============================================================
#  SIMULATED ANNEALING
# ============================================================

def simulated_annealing(n, m, direction,
                        T_start=3.0, T_end=0.001,
                        cooling=0.99, max_steps=300,
                        initial_graph=None):

    G = Graph(initial_graph)

    best_G = G.copy()
    best_score = subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):
        new_G = mutate_graph(current_G)
        new_score = subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G = new_G
            current_score = new_score
        else:
            if pyrandom.random() < exp(-delta/T):
                current_G = new_G
                current_score = new_score

        improved = (
            (direction == "min" and current_score < best_score) or
            (direction == "max" and current_score > best_score)
        )

        if improved:
            best_G = current_G.copy()
            best_score = current_score

        T *= cooling
        if T < T_end:
            break

    return best_G, best_score


# ============================================================
#  GONILNA FUNKCIJA
# ============================================================

def compute_all_for_n9(direction):

    n = 9
    mu_max_prev = 21
    mu_max_n9 = 28

    for mu in range(0, mu_max_n9 + 1):

        print(f"Obdelujem: n=9, µ={mu}")

        if mu <= mu_max_prev:
            G8 = load_G(8, mu, direction)
            Gstart = add_one_vertex_and_connect(G8, direction)
        else:
            Gprev = load_G(9, mu-1, direction)
            Gstart = add_one_edge(Gprev)

        m = mu + n - 1

        best_graph, best_value = simulated_annealing(
            n=n,
            m=m,
            direction=direction,
            initial_graph=Gstart
        )

        print(f"=== Rezultati za n={n}, µ={mu} ===")
        print("Best value:", best_value)
        print("graph6:", best_graph.graph6_string())

        save_to_csv(n, mu, best_graph, best_value, direction)


# ============================================================
#  START
# ============================================================

compute_all_for_n9(direction="max")


Spodnja koda dela SA za n=9 iz random zacetnimi priblizki. Shranjuje v 1del_SA_n9_random.csv

In [ ]:

import random as pyrandom
from sage.all import *
import pandas as pd
import os
import time

# ============================================================
#  SUBPATH NUMBER (OPTIMIZIRANO)
# ============================================================

def subpath_number(G):
    V = list(G.vertices())
    idx = {v: i for i, v in enumerate(V)}
    n = len(V)

    counts = [[0] * n for _ in range(n)]

    for s in V:
        s_idx = idx[s]

        def dfs(u, visited):
            u_idx = idx[u]
            counts[s_idx][u_idx] += 1

            for nei in G.neighbors(u):
                if nei not in visited:
                    visited.add(nei)
                    dfs(nei, visited)
                    visited.remove(nei)

        visited = set([s])
        dfs(s, visited)

    total = 0
    for i in range(n):
        for j in range(i, n):
            total += counts[i][j]

    return total


# ============================================================
#  SHARANJE OPTIMUMA v CSV (popolnoma popravljeno)
# ============================================================

def save_to_csv(n, mu, graph, score, direction):
    filename = "1del_SA_n9_random.csv"

    g6 = graph.graph6_string()

    # Če CSV ne obstaja → kreiraj
    if not os.path.exists(filename):
        df = pd.DataFrame(columns=[
            "n",
            "µ(G)",
            "min p_n(G)",
            "max p_n(G)",
            "graph6_min",
            "graph6_max"
        ])
        df.to_csv(filename, index=False)

    df = pd.read_csv(filename, encoding="utf-8")

    # poskrbimo, da stolpci obstajajo (če imaš kak star CSV)
    for col in ["min p_n(G)", "max p_n(G)", "graph6_min", "graph6_max"]:
        if col not in df.columns:
            df[col] = pd.NA

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    n_int = int(n)
    mu_int = int(mu)

    mask = (df["n"] == n_int) & (df["µ(G)"] == mu_int)

    # 1) Če vrstice NI → nova
    if not mask.any():
        new_row = {
            "n": n_int,
            "µ(G)": mu_int,
            "min p_n(G)": score if direction == "min" else pd.NA,
            "max p_n(G)": score if direction == "max" else pd.NA,
            "graph6_min": g6 if direction == "min" else pd.NA,
            "graph6_max": g6 if direction == "max" else pd.NA
        }

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        df.to_csv(filename, index=False)
        return

    # 2) Če vrstica OBSTAJA → update
    idx = df.index[mask].tolist()[0]

    if direction == "min":
        old = df.at[idx, "min p_n(G)"]

        if pd.isna(old) or old == "":
            # še nič ni zapisano → zapišemo
            df.at[idx, "min p_n(G)"] = score
            df.at[idx, "graph6_min"] = g6
        else:
            # že obstaja številka → zamenjamo samo, če je nova boljša (manjša)
            if score < float(old):
                df.at[idx, "min p_n(G)"] = score
                df.at[idx, "graph6_min"] = g6

    else:  # direction == "max"
        old = df.at[idx, "max p_n(G)"]

        if pd.isna(old) or old == "":
            df.at[idx, "max p_n(G)"] = score
            df.at[idx, "graph6_max"] = g6
        else:
            if score > float(old):
                df.at[idx, "max p_n(G)"] = score
                df.at[idx, "graph6_max"] = g6

    df.to_csv(filename, index=False)


# ============================================================
#  MUTATE GRAPH
# ============================================================

def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H

def random_connected_graph(n, m):
    while True:
        G = graphs.RandomGNM(n, m)
        if G.is_connected():
            return G



# ============================================================
#  SIMULATED ANNEALING
# ============================================================

def simulated_annealing(n, m, direction,
                        T_start=6.0, T_end=0.005,
                        cooling=0.99, max_steps=300,
                        initial_graph=None):

    G = Graph(initial_graph)

    best_G = G.copy()
    best_score = subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):
        new_G = mutate_graph(current_G)
        new_score = subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G = new_G
            current_score = new_score
        else:
            if pyrandom.random() < exp(-delta/T):
                current_G = new_G
                current_score = new_score

        improved = (
            (direction == "min" and current_score < best_score) or
            (direction == "max" and current_score > best_score)
        )

        if improved:
            best_G = current_G.copy()
            best_score = current_score

        T *= cooling
        if T < T_end:
            break

    return best_G, best_score


# ============================================================
#  GONILNA FUNKCIJA
# ============================================================

def compute_all_for_n9(direction):

    n = 9
    mu_max_n9 = 28

    for mu in range(26, mu_max_n9 + 1):

        print(f"Obdelujem: n=9, µ={mu}")

        m = mu + n - 1

        Gstart = random_connected_graph(n, m)

        best_graph, best_value = simulated_annealing(
            n=n,
            m=m,
            direction=direction,
            initial_graph=Gstart
        )

        print(f"=== Rezultati za n={n}, µ={mu} ===")
        print("Best value:", best_value)
        print("graph6:", best_graph.graph6_string())

        save_to_csv(n, mu, best_graph, best_value, direction)




# ============================================================
#  START
# ============================================================

compute_all_for_n9(direction="max")


In [10]:

import random as pyrandom
from sage.all import *
import pandas as pd
import os
import time

# ============================================================
#  SUBPATH NUMBER (OPTIMIZIRANO)
# ============================================================

def subpath_number(G):
    V = list(G.vertices())
    idx = {v: i for i, v in enumerate(V)}
    n = len(V)

    counts = [[0] * n for _ in range(n)]

    for s in V:
        s_idx = idx[s]

        def dfs(u, visited):
            u_idx = idx[u]
            counts[s_idx][u_idx] += 1

            for nei in G.neighbors(u):
                if nei not in visited:
                    visited.add(nei)
                    dfs(nei, visited)
                    visited.remove(nei)

        visited = set([s])
        dfs(s, visited)

    total = 0
    for i in range(n):
        for j in range(i, n):
            total += counts[i][j]

    return total


# ============================================================
#  SHARANJE OPTIMUMA v CSV (popolnoma popravljeno)
# ============================================================

def save_to_csv(n, mu, graph, score, direction):
    filename = "1del_SA_n9_priblizki8.csv"

    g6 = graph.graph6_string()

    # Če CSV ne obstaja → kreiraj
    if not os.path.exists(filename):
        df = pd.DataFrame(columns=[
            "n",
            "µ(G)",
            "min p_n(G)",
            "max p_n(G)",
            "graph6_min",
            "graph6_max"
        ])
        df.to_csv(filename, index=False)

    df = pd.read_csv(filename, encoding="utf-8")

    # poskrbimo, da stolpci obstajajo (če imaš kak star CSV)
    for col in ["min p_n(G)", "max p_n(G)", "graph6_min", "graph6_max"]:
        if col not in df.columns:
            df[col] = pd.NA

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    n_int = int(n)
    mu_int = int(mu)

    mask = (df["n"] == n_int) & (df["µ(G)"] == mu_int)

    # 1) Če vrstice NI → nova
    if not mask.any():
        new_row = {
            "n": n_int,
            "µ(G)": mu_int,
            "min p_n(G)": score if direction == "min" else pd.NA,
            "max p_n(G)": score if direction == "max" else pd.NA,
            "graph6_min": g6 if direction == "min" else pd.NA,
            "graph6_max": g6 if direction == "max" else pd.NA
        }

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        df.to_csv(filename, index=False)
        return

    # 2) Če vrstica OBSTAJA → update
    idx = df.index[mask].tolist()[0]

    if direction == "min":
        old = df.at[idx, "min p_n(G)"]

        if pd.isna(old) or old == "":
            # še nič ni zapisano → zapišemo
            df.at[idx, "min p_n(G)"] = score
            df.at[idx, "graph6_min"] = g6
        else:
            # že obstaja številka → zamenjamo samo, če je nova boljša (manjša)
            if score < float(old):
                df.at[idx, "min p_n(G)"] = score
                df.at[idx, "graph6_min"] = g6

    else:  # direction == "max"
        old = df.at[idx, "max p_n(G)"]

        if pd.isna(old) or old == "":
            df.at[idx, "max p_n(G)"] = score
            df.at[idx, "graph6_max"] = g6
        else:
            if score > float(old):
                df.at[idx, "max p_n(G)"] = score
                df.at[idx, "graph6_max"] = g6

    df.to_csv(filename, index=False)



# ============================================================
#  MUTATE GRAPH
# ============================================================

def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H


# ============================================================
#  DODAJ 1 VOZLIŠČE
# ============================================================

def add_one_vertex_and_connect(G_old, direction):
    H = Graph(G_old)

    verts = H.vertices()
    new_v = max(verts) + 1
    H.add_vertex(new_v)

    degrees = {v: H.degree(v) for v in H.vertices() if v != new_v}

    if direction == "min":
        anchor = min(degrees, key=degrees.get)
    else:
        anchor = max(degrees, key=degrees.get)

    H.add_edge(anchor, new_v)
    return H


# ============================================================
#  DODAJ 1 POVEZAVO (najbližji non-edge)
# ============================================================

def add_one_edge(G_old):
    H = Graph(G_old)

    V = H.vertices()
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if not H.has_edge(u, v):
                H.add_edge(u, v)
                return H

    raise ValueError("Graf je že poln.")


# ============================================================
#  LOAD G(n,µ) IZ TVOJEGA CSV — POPRAVLJENO
# ============================================================

def load_G(n_val, mu_val, direction):
    df = pd.read_csv("1del_SA_n9_priblizki8.csv", encoding="utf-8")

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    # Konverzija Sage Integer → Python int
    n_val = int(n_val)
    mu_val = int(mu_val)

    subset = df[df["n"] == n_val]
    if subset.empty:
        raise ValueError(f"Ni vrstic za n={n_val}")

    row = subset[subset["µ(G)"] == mu_val]
    if row.empty:
        raise ValueError(f"Ni vrstice za n={n_val}, µ={mu_val}")

    # KLJUČNO — prisili Python int indeks
    row = row.iloc[int(0)]      # ← EDINA PRAVA OBLIKA

    # graph6
    if direction == "min":
        g6 = str(row["graph6_min"]).strip()
    else:
        g6 = str(row["graph6_max"]).strip()

    if g6 == "":
        raise ValueError(f"Prazni graph6 zapis pri n={n_val}, µ={mu_val}")

    return Graph(g6)



# ============================================================
#  SIMULATED ANNEALING
# ============================================================

def simulated_annealing(n, m, direction,
                        T_start=3.0, T_end=0.001,
                        cooling=0.99, max_steps=300,
                        initial_graph=None):

    G = Graph(initial_graph)

    best_G = G.copy()
    best_score = subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):
        new_G = mutate_graph(current_G)
        new_score = subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G = new_G
            current_score = new_score
        else:
            if pyrandom.random() < exp(-delta/T):
                current_G = new_G
                current_score = new_score

        improved = (
            (direction == "min" and current_score < best_score) or
            (direction == "max" and current_score > best_score)
        )

        if improved:
            best_G = current_G.copy()
            best_score = current_score

        T *= cooling
        if T < T_end:
            break

    return best_G, best_score


# ============================================================
#  GONILNA FUNKCIJA
# ============================================================

def compute_all_for_n9(direction):

    n = 9
    mu_max_prev = 21
    mu_max_n9 = 28

    for mu in range(0, mu_max_n9 + 1):

        print(f"Obdelujem: n=9, µ={mu}")

        if mu <= mu_max_prev:
            G8 = load_G(8, mu, direction)
            Gstart = add_one_vertex_and_connect(G8, direction)
        else:
            Gprev = load_G(9, mu-1, direction)
            Gstart = add_one_edge(Gprev)

        m = mu + n - 1

        best_graph, best_value = simulated_annealing(
            n=n,
            m=m,
            direction=direction,
            initial_graph=Gstart
        )

        print(f"=== Rezultati za n={n}, µ={mu} ===")
        print("Best value:", best_value)
        print("graph6:", best_graph.graph6_string())

        save_to_csv(n, mu, best_graph, best_value, direction)


# ============================================================
#  START
# ============================================================

compute_all_for_n9(direction="max")


Obdelujem: n=9, µ=0
=== Rezultati za n=9, µ=0 ===
Best value: 45
graph6: H??FwCA
Obdelujem: n=9, µ=1
=== Rezultati za n=9, µ=1 ===
Best value: 81
graph6: HOOHe?S
Obdelujem: n=9, µ=2


/tmp/ipykernel_5130/367253232.py:113: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '45' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "max p_n(G)"] = score
/tmp/ipykernel_5130/367253232.py:117: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '81' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "max p_n(G)"] = score


=== Rezultati za n=9, µ=2 ===
Best value: 133
graph6: H?st@@@
Obdelujem: n=9, µ=3
=== Rezultati za n=9, µ=3 ===
Best value: 221
graph6: HHISODD
Obdelujem: n=9, µ=4
=== Rezultati za n=9, µ=4 ===
Best value: 372
graph6: HCSqN?g
Obdelujem: n=9, µ=5
=== Rezultati za n=9, µ=5 ===
Best value: 615
graph6: HCPVCWw
Obdelujem: n=9, µ=6
=== Rezultati za n=9, µ=6 ===
Best value: 955
graph6: HSSRbIK
Obdelujem: n=9, µ=7
=== Rezultati za n=9, µ=7 ===
Best value: 1434
graph6: H@tfAeD
Obdelujem: n=9, µ=8
=== Rezultati za n=9, µ=8 ===
Best value: 2135
graph6: HamTaSd
Obdelujem: n=9, µ=9
=== Rezultati za n=9, µ=9 ===
Best value: 3178
graph6: HqXcxiH
Obdelujem: n=9, µ=10


/tmp/ipykernel_5130/367253232.py:117: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3178' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "max p_n(G)"] = score


=== Rezultati za n=9, µ=10 ===
Best value: 4757
graph6: HFdnBQF
Obdelujem: n=9, µ=11
=== Rezultati za n=9, µ=11 ===
Best value: 6517
graph6: HiP\\qw
Obdelujem: n=9, µ=12
=== Rezultati za n=9, µ=12 ===
Best value: 8806
graph6: HXulalU
Obdelujem: n=9, µ=13


...

=== Rezultati za n=9, µ=26 ===
Best value: 321631
graph6: H~~^~z~
Obdelujem: n=9, µ=27


/tmp/ipykernel_5130/367253232.py:113: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '321631' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "max p_n(G)"] = score


=== Rezultati za n=9, µ=27 ===
Best value: 397308
graph6: H~~~~z~
Obdelujem: n=9, µ=28


/tmp/ipykernel_5130/367253232.py:113: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '397308' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "max p_n(G)"] = score


KeyboardInterrupt: 